In [1]:
# import the required libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
# Read the text file
with open(r"C:\Users\vedhi\Downloads\book\sherlock-holm.es_stories_plain-text_advs.txt", encoding='utf-8') as file:
    text = file.read()

In [3]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [4]:
# create input-output pairs by splitting the text into sequences of tokens and forming n-grams from the sequences
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [5]:
# pad the input sequences to have equal length
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [6]:
# split the sequences into input and output
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [7]:
# convert the output to one-hot encode vectors
y = np.array(tf.keras.utils.to_categorical(y, num_classes = total_words))

In [8]:
# build a neural network architecture to train the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length = max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation = 'softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2,208,800
Trainable params: 2,208,800
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
# compile the model
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X, y, epochs = 10, verbose = 1)

Epoch 1/10
3010/3010 [==============================] - 190s 61ms/step - loss: 3.3811 - accuracy: 0.3135
Epoch 2/10
3010/3010 [==============================] - 185s 61ms/step - loss: 3.0254 - accuracy: 0.3594
Epoch 3/10
3010/3010 [==============================] - 186s 62ms/step - loss: 2.7693 - accuracy: 0.4059
Epoch 4/10
3010/3010 [==============================] - 186s 62ms/step - loss: 2.5661 - accuracy: 0.4418
Epoch 5/10
3010/3010 [==============================] - 187s 62ms/step - loss: 2.3855 - accuracy: 0.4763
Epoch 6/10
3010/3010 [==============================] - 185s 62ms/step - loss: 2.2305 - accuracy: 0.5076
Epoch 7/10
3010/3010 [==============================] - 187s 62ms/step - loss: 2.0890 - accuracy: 0.5376
Epoch 8/10
3010/3010 [==============================] - 182s 60ms/step - loss: 1.9607 - accuracy: 0.5637
Epoch 9/10
3010/3010 [==============================] - 182s 60ms/step - loss: 1.8438 - accuracy: 0.5889
Epoch 10/10
3010/3010 [==============================] 

In [18]:
seed_text = "I will leave if they"
next_words = 25

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 40ms/step
I will leave if they will soon set it over this matter for i have a soul either out to them than when we came through hatherley singular morning train


In [19]:
print(seed_text)

I will leave if they will soon set it over this matter for i have a soul either out to them than when we came through hatherley singular morning train
